### LSTM model tuner

* Define functions
* try to find the best hyperparameters
* Train the model

Importing libraries

In [57]:
import pandas as pd
from matplotlib import pyplot

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Input
from keras.callbacks import TensorBoard
from keras.layers import Dropout
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

from math import sqrt
from numpy import concatenate

Define functions

In [58]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [59]:
tf.compat.v1.keras.layers.CuDNNLSTM

def lstm_model_builder(hp):

    model = Sequential()

    model.add(Input(
        shape=(train_x.shape[1],train_x.shape[2])
    ))
    
    # model.add(LSTM(hp.Int('input_unit',
    #                     min_value=32,
    #                     max_value=512,
    #                     step=32), 
    #                     return_sequences=True, 
    #                     input_shape=(train_x.shape[1],train_x.shape[2])))

    for i in range(hp.Int('n_layers', 1, 3)):

        model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=False))
        model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=512,step=32)))
        model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
        model.add(Dense(train_y.shape[1], activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
        model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mse'])
        
    return model

Load data

In [60]:
df = pd.read_parquet("../data/processed/new-avc.parquert")
df.head()

,date,co_ppb,o3_ppb,pm25_ugm3,so2_ugm3,temperatura_c,umidade_relativa_percentual,precipitacao_mmdia,focos_queimada,respiratory,avc
0,2013-01-01,135.40,11.125,8.175,0.875,26.450,81.75,2.0,0.0,6.0,1.0
1,2013-01-02,155.05,10.925,11.675,1.000,26.400,82.00,2.0,0.0,8.0,0.0
2,2013-01-03,137.40,12.225,8.800,0.825,27.400,81.25,0.0,0.0,12.0,1.0
3,2013-01-04,128.30,13.750,11.650,0.925,27.750,76.25,1.0,0.0,9.0,0.0
4,2013-01-05,128.25,10.950,7.800,1.075,26.625,84.75,9.0,0.0,10.0,1.0


Normalize features

In [61]:
df = df.set_index("date")

In [62]:
df = df.drop(columns=["avc"])

In [63]:
values = df.values

In [64]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

Apply series_to_supervised

In [65]:
n_lags = 3

In [66]:
df_sl = series_to_supervised(scaled, n_lags, 1)

Drop columns I don't want to predict

In [67]:
df_sl.drop(df_sl.columns[[27, 28, 29, 30, 31, 32, 33, 34]], axis=1, inplace=True)

Define and fit model

Split into train and test sets

In [68]:
values = df_sl.values
train_set, test_set = train_test_split(
    df_sl, test_size=0.2, shuffle=False
)

In [69]:
train_X, train_Y = train_set.drop("var9(t)", axis=1), train_set.drop(train_set.columns[:27], axis=1)
test_X, test_Y = test_set.drop("var9(t)", axis=1), test_set.drop(test_set.columns[:27], axis=1)

Reshape input to be 3D [samples, timesteps, features]

In [70]:
train_x, test_x = train_X.values, test_X.values
train_y, test_y = train_Y.values, test_Y.values

In [71]:
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

In [72]:
tuner= RandomSearch(
        lstm_model_builder,
        objective='mse',
        max_trials=2,
        executions_per_trial=1
        )

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


ValueError: Input 0 of layer "lstm_4" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 32)

In [ ]:
tuner.search(
        x=train_x,
        y=train_y,
        epochs=20,
        batch_size=128,
        validation_data=(test_x,test_y),
)


Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
320               |384               |input_unit
3                 |1                 |n_layers
416               |64                |lstm_0_units
352               |64                |layer_2_neurons
0.2               |0.3               |Dropout_rate
sigmoid           |relu              |dense_activation



Traceback (most recent call last):
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_2429/1863909391.py", line 20, in lstm_model_builder
    model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras/engine/input_spec.p

Invalid model 0/5


Traceback (most recent call last):
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_2429/1863909391.py", line 20, in lstm_model_builder
    model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras/engine/input_spec.p

Invalid model 1/5


...

Invalid model 3/5


Traceback (most recent call last):
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_2429/1863909391.py", line 20, in lstm_model_builder
    model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras/engine/input_spec.p

Invalid model 4/5
Invalid model 5/5


Traceback (most recent call last):
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_2429/1863909391.py", line 20, in lstm_model_builder
    model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/matheus/miniconda3/envs/tg/lib/python3.10/site-packages/keras/engine/input_spec.p

RuntimeError: Too many failed attempts to build model.